#Connect to sharepoint

In [9]:
!pip install office365
!pip install Office365-REST-Python-Client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.4/185.4 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.client_credential import ClientCredential
from office365.sharepoint.files.file import File

####inputs########
# This will be the URL that points to your sharepoint site.
# Make sure you change only the parts of the link that start with "Your"
url_shrpt = 'https://ihuedu.sharepoint.com/sites/EDYTEProject2023/'
username_shrpt = '###############'
password_shrpt = '###############'
folder_files_url_shrpt = '/sites/EDYTEProject2023/Shared%20Documents/General/wp5_data/analysis_exports/keywords/keywords_topics/'
folder_analysis_url_shrpt = '/sites/EDYTEProject2023/Shared%20Documents/General/wp5_data/analysis_exports/'

In [11]:
###Authentication###For authenticating into your sharepoint site###
ctx_auth = AuthenticationContext(url_shrpt)
if ctx_auth.acquire_token_for_user(username_shrpt, password_shrpt):
  ctx = ClientContext(url_shrpt, ctx_auth)
  web = ctx.web
  ctx.load(web)
  ctx.execute_query()
  print('Authenticated into sharepoint as: ',web.properties['Title'])

else:
  print(ctx_auth.get_last_error())
############################

Authenticated into sharepoint as:  EDYTE Project 2023


In [12]:
####Function for extracting the file names of a folder in sharepoint###
###If you want to extract the folder names instead of file names, you have to change "sub_folders = folder.files" to "sub_folders = folder.folders" in the below function

global print_folder_contents
def print_folder_contents(ctx, folder_url):
    try:

        folder = ctx.web.get_folder_by_server_relative_url(folder_url)
        fold_names = []
        sub_folders = folder.files #Replace files with folders for getting list of folders
        ctx.load(sub_folders)
        ctx.execute_query()

        for s_folder in sub_folders:

            fold_names.append(s_folder.properties["Name"])

        return fold_names

    except Exception as e:
        print('Problem printing out library contents: ', e)
######################################################

# Call the function by giving your folder URL as input
filelist_shrpt=print_folder_contents(ctx,folder_files_url_shrpt)
#Print the list of files present in the folder
print(filelist_shrpt)

['process_evidences_keywords_topic.csv', 'steps_digital_keywords_topic.csv', 'process_keywords_topic.csv', 'process_conditions_keywords_topic.csv', 'process_rules_keywords_topic.csv', 'steps_keywords_topic.csv']


## ORGs

In [6]:
import requests
import pandas as pd

def get_data_from_page(url):
    response = requests.get(url)
    json_data = response.json()
    data_list = []
    for item in json_data["data"]:
        title_el = item["title"].get("el", "")
        title_en = item["title"].get("en", "")
        data_list.append({
            "id": item["id"],
            "parent_id": item["parent_id"],
            "descendants_count": item["descendants_count"],
            "title_el": title_el,
            "title_en": title_en,
            "link": item["link"],
        })
    return data_list, json_data.get("next_page", 0)

def fetch_data_until_no_next_page(base_url):
    all_data = []
    page_number = 1
    while True:
        url = f"{base_url}?page={page_number}"
        data_list, next_page = get_data_from_page(url)
        all_data.extend(data_list)
        if not next_page:
            break
        page_number += 1
    return all_data

In [13]:
base_url = "https://api.digigov.grnet.gr/v1/organizations"
all_data_list = fetch_data_until_no_next_page(base_url)
orgs = pd.DataFrame(all_data_list)

In [14]:
# Display the DataFrame
orgs

,id,parent_id,descendants_count,title_el,title_en,link
0,11698,40151,0,"ΑΙΑΝΤΕΙΟΣ"" ΔΗΜΟΣΙΑ ΒΙΒΛΙΟΘΗΚΗ ΑΤΑΛΑΝΤΗΣ ΦΘΙΩΤΙ...",AIANTIOS PUBLIC LIBRARY OF MUNICIPALITY OF ATA...,/v1/organization?id=11698
1,23462,99487,0,ΥΠΟΘΗΚΟΦΥΛΑΚΕΙΟ ΘΕΣΣΑΛΟΝΙΚΗΣ,LOCAL MORTGAGE REGISTRY OF THESSALONIKI,/v1/organization?id=23462
2,58136,58242,0,ΔΗΜΟΤΙΚΗ ΚΟΙΝΩΦΕΛΗΣ ΕΠΙΧΕΙΡΗΣΗ ΠΥΔΝΑΣ ΚΟΛΙΝΔΡΟΥ,MUNICIPAL UTILITY COMPANY OF PYDNA KOLINDROS,/v1/organization?id=58136
3,68955,95060,0,ΚΕΝΤΡΟ ΜΕΡΙΜΝΑΣ ΚΑΙ ΑΛΛΗΛΕΓΓΥΗΣ ΔΗΜΟΥ ΚΟΜΟΤΗΝΗ...,CARE AND SOLIDARITY CENTER OF KOMOTINI MUNICIP...,/v1/organization?id=68955
4,34394,54032,0,ΦΟΡΕΑΣ ΔΙΑΧΕΙΡΙΣΗΣ ΥΓΡΟΤΟΠΩΝ ΚΟΤΥΧΙΟΥ ΣΤΡΟΦΥΛΙΑΣ,KOTYCHI – STROFYLIA WETLANDS MANAGEMENT BODY,/v1/organization?id=34394
...,...,...,...,...,...,...
4387,81939,45210,0,ΣΧΟΛΙΚΗ ΕΠΙΤΡΟΠΗ ΠΡΩΤΟΒΑΘΜΙΑΣ ΕΚΠΑΙΔΕΥΣΗΣ ΔΗΜΟ...,SCHOOL COMMITTEE FOR PRIMARY EDUCATION - MUNIC...,/v1/organization?id=81939
4388,17761,58242,0,ΚΥΠΑΡΙΣΣΟΠΟΥΛΕΙΟ ΚΛΗΡΟΔΟΤΗΜΑ ΔΗΜΟΥ ΠΥΔΝΑΣ ΚΟΛΙ...,KYPARISSOPOULEIO LEGACY OF THE MUNICIPALITY OF...,/v1/organization?id=17761
4389,29671,40151,0,ΙΕΡΑ ΜΗΤΡΟΠΟΛΙΣ ΠΕΤΡΑΣ & ΧΕΡΡΟΝΗΣΟΥ,METROPOLIS OF PETRA AND CHERONISSOS,/v1/organization?id=29671
4390,26881,76877,0,ΑΠΟΚΟΡΩΝΑΣ ΑΝΑΠΤΥΞΙΑΚΗ ΕΤΑΙΡΕΙΑ -ΟΤΑ,DEVELOPMENT AGENCY - APOKORONAS,/v1/organization?id=26881


# Analysis

## Orgs

In [45]:
import requests
import pandas as pd

def get_data_from_page(url):
    response = requests.get(url)
    json_data = response.json()
    data_list = []
    for item in json_data["data"]:
        title_el = item["title"].get("el", "")
        title_en = item["title"].get("en", "")
        data_list.append({
            "id": item["id"],
            "parent_id": item["parent_id"],
            "descendants_count": item["descendants_count"],
            "title_el": title_el,
            "title_en": title_en,
            "link": item["link"],
        })
    return data_list, json_data.get("next_page", 0)

def fetch_data_until_no_next_page(base_url):
    all_data = []
    page_number = 1
    while True:
        url = f"{base_url}?page={page_number}"
        data_list, next_page = get_data_from_page(url)
        all_data.extend(data_list)
        if not next_page:
            break
        page_number += 1
    return all_data

In [47]:
base_url = "https://api.digigov.grnet.gr/v1/organizations"
all_data_list = fetch_data_until_no_next_page(base_url)
orgs = pd.DataFrame(all_data_list)

In [48]:
orgs

,id,parent_id,descendants_count,title_el,title_en,link
0,11698,40151,0,"ΑΙΑΝΤΕΙΟΣ"" ΔΗΜΟΣΙΑ ΒΙΒΛΙΟΘΗΚΗ ΑΤΑΛΑΝΤΗΣ ΦΘΙΩΤΙ...",AIANTIOS PUBLIC LIBRARY OF MUNICIPALITY OF ATA...,/v1/organization?id=11698
1,23462,99487,0,ΥΠΟΘΗΚΟΦΥΛΑΚΕΙΟ ΘΕΣΣΑΛΟΝΙΚΗΣ,LOCAL MORTGAGE REGISTRY OF THESSALONIKI,/v1/organization?id=23462
2,58136,58242,0,ΔΗΜΟΤΙΚΗ ΚΟΙΝΩΦΕΛΗΣ ΕΠΙΧΕΙΡΗΣΗ ΠΥΔΝΑΣ ΚΟΛΙΝΔΡΟΥ,MUNICIPAL UTILITY COMPANY OF PYDNA KOLINDROS,/v1/organization?id=58136
3,68955,95060,0,ΚΕΝΤΡΟ ΜΕΡΙΜΝΑΣ ΚΑΙ ΑΛΛΗΛΕΓΓΥΗΣ ΔΗΜΟΥ ΚΟΜΟΤΗΝΗ...,CARE AND SOLIDARITY CENTER OF KOMOTINI MUNICIP...,/v1/organization?id=68955
4,34394,54032,0,ΦΟΡΕΑΣ ΔΙΑΧΕΙΡΙΣΗΣ ΥΓΡΟΤΟΠΩΝ ΚΟΤΥΧΙΟΥ ΣΤΡΟΦΥΛΙΑΣ,KOTYCHI – STROFYLIA WETLANDS MANAGEMENT BODY,/v1/organization?id=34394
...,...,...,...,...,...,...
4387,81939,45210,0,ΣΧΟΛΙΚΗ ΕΠΙΤΡΟΠΗ ΠΡΩΤΟΒΑΘΜΙΑΣ ΕΚΠΑΙΔΕΥΣΗΣ ΔΗΜΟ...,SCHOOL COMMITTEE FOR PRIMARY EDUCATION - MUNIC...,/v1/organization?id=81939
4388,17761,58242,0,ΚΥΠΑΡΙΣΣΟΠΟΥΛΕΙΟ ΚΛΗΡΟΔΟΤΗΜΑ ΔΗΜΟΥ ΠΥΔΝΑΣ ΚΟΛΙ...,KYPARISSOPOULEIO LEGACY OF THE MUNICIPALITY OF...,/v1/organization?id=17761
4389,29671,40151,0,ΙΕΡΑ ΜΗΤΡΟΠΟΛΙΣ ΠΕΤΡΑΣ & ΧΕΡΡΟΝΗΣΟΥ,METROPOLIS OF PETRA AND CHERONISSOS,/v1/organization?id=29671
4390,26881,76877,0,ΑΠΟΚΟΡΩΝΑΣ ΑΝΑΠΤΥΞΙΑΚΗ ΕΤΑΙΡΕΙΑ -ΟΤΑ,DEVELOPMENT AGENCY - APOKORONAS,/v1/organization?id=26881


## Output Registries

In [18]:
def fetch_json_data(url):
    response = requests.get(url)
    json_data = response.json()
    return json_data

def create_dataframe_from_json(json_data):
    data_list = json_data["data"]
    df_data = [{"id": item["id"], "title": item["title"]["el"]} for item in data_list]
    df = pd.DataFrame(df_data)
    return df

In [19]:
url = "https://api.digigov.grnet.gr/v1/registry/all/"
json_data = fetch_json_data(url)
output_registries = create_dataframe_from_json(json_data)

In [20]:
output_registries

,id,title
0,279107,ATLAS
1,413792,Back Office
2,207565,BI FORMS
3,604960,Datacenter
4,431208,E-doatap
...,...,...
753,746061,ΧΟΡΗΓΗΣΗ ΠΙΣΤΟΠΟΙΗΤΙΚΟΥ ΟΡΘΗΣ ΑΣΚΗΣΗΣ ΙΑΤΡΙΚΟΥ...
754,992549,Χορήγηση της λίστας του άρθρου 13
755,136788,ΧΟΡΗΓΗΣΗ ΤΙΤΛΟΥ ΙΑΤΡΙΚΗΣ ΕΙΔΙΚΟΤΗΤΑΣ ΑΠΟ Ε.Ε. ...
756,296834,Ψηφιακή εφαρμογή της απογραφής του Ελληνικού Δ...


## provision_org

In [28]:
def fetch_json_data(url):
    response = requests.get(url)
    json_data = response.json()
    return json_data

def create_dataframe_from_json(json_data):
    data_list = json_data["data"]
    df_data = []
    for item in data_list:
        item_dict = {"id": item["id"], "title": item["title"]["el"]}
        if "url" in item:
            item_dict["url"] = item["url"]
        df_data.append(item_dict)
    df = pd.DataFrame(df_data)
    return df

In [29]:
url = "https://api.digigov.grnet.gr/v1/registries/evidence/taxonomy/provision_org_gro/"
json_data = fetch_json_data(url)
provision_org = create_dataframe_from_json(json_data)

In [30]:
provision_org

,id,title,url
0,1035,myAADElive,https://myaadelive.gov.gr/dsae2/govdesk/faces/...
1,8579,myConsulLive,https://www.gov.gr/ipiresies/polites-kai-kathe...
2,1249,myDIMOSlive,https://www.gsis.gr/myDimosLive
3,3954,myDYPAlive,https://www.gov.gr/ipiresies/polites-kai-kathe...
4,5493,myEFKAlive,https://www.gsis.gr/polites-epiheiriseis/stoih...
5,8477,myEGDIXlive,https://www.gov.gr/ipiresies/polites-kai-kathe...
6,8256,myKEPlive,https://www.gov.gr/ipiresies/polites-kai-kathe...
7,3621,myKEPlive (μόνο πληροφορίες),https://www.gov.gr/ipiresies/polites-kai-kathe...
8,3450,myKTIMATOLOGIOlive,https://www.gsis.gr/polites-epiheiriseis/stoih...
9,4721,mySynigoroslive,https://mysynigoros.live.gov.gr/dsae2/govdesks...


## evidence_type

In [32]:
def fetch_json_data(url):
    response = requests.get(url)
    json_data = response.json()
    return json_data

def create_dataframe_from_json(json_data):
    data_list = json_data["data"]
    df_data = [{"id": item["id"], "title": item["title"]["el"]} for item in data_list]
    df = pd.DataFrame(df_data)
    return df

In [35]:
url = "https://api.digigov.grnet.gr/v1/registries/evidence/all"
json_data = fetch_json_data(url)
evidence_type = create_dataframe_from_json(json_data)

In [36]:
evidence_type

,id,title
0,4627,Άδεια
1,9960,Άδεια άσκησης επαγγέλματος
2,4178,Άδεια διαμονής
3,2243,Άδεια δόμησης
4,4475,Άδεια Εγκατάστασης
...,...,...
196,3353,Χημική Ανάλυση
197,9481,Χρονοδιάγραμμα κατασκευής έργου
198,4673,Ψηφιακή άδεια οδήγησης
199,4128,Ψηφιακό αρχείο


## Provision Org Owner Directory

In [53]:
from concurrent.futures import ThreadPoolExecutor

In [55]:
def get_data_from_page(url):
    response = requests.get(url)
    json_data = response.json()
    data_list = []
    for item in json_data["data"]:
        title_el = item["title"].get("el", "")
        data_list.append({
            "id": item["id"],
            "parent_id": item["parent_id"],
            "descendants_count": item["descendants_count"],
            "title_el": title_el,
            "link": item["link"],
        })
    return data_list, json_data.get("next_page", 0)

def fetch_data_until_no_next_page(base_url):
    all_data = []
    page_number = 1
    with ThreadPoolExecutor(max_workers=50) as executor:  # You can adjust the number of workers
      while True:
          url = f"{base_url}?page={page_number}"
          data_list, next_page = get_data_from_page(url)
          all_data.extend(data_list)
          if not next_page:
              break
          page_number += 1
    return all_data

In [56]:
base_url = "https://api.digigov.grnet.gr/v1/units"
all_data_list = fetch_data_until_no_next_page(base_url)
provision_org_owner_directory = pd.DataFrame(all_data_list)

In [57]:
provision_org_owner_directory

,id,parent_id,descendants_count,title_el,link
0,492159,19381,0,ΤΜΗΜΑ ΠΡΟΣΤΑΣΙΑΣ ΚΑΙ ΠΡΟΑΓΩΓΗΣ ΤΗΣ ΔΗΜΟΣΙΑΣ ΥΓ...,/v1/unit?id=492159
1,991278,90268,0,ΥΔΡΑΥΛΙΚΟ ΤΜΗΜΑ,/v1/unit?id=991278
2,623977,56492,0,ΓΡΑΦΕΙΟ ΜΕΛΕΤΩΝ ΚΑΙ ΣΥΜΒΑΣΕΩΝ,/v1/unit?id=623977
3,643735,71951,0,ΑΥΤΟΤΕΛΕΣ ΓΡΑΦΕΙΟ ΔΙΟΙΚΗΤΙΚΗΣ ΒΟΗΘΕΙΑΣ,/v1/unit?id=643735
4,778764,95618,0,ΤΜΗΜΑ ΕΣΩΤΕΡΙΚΗΣ ΑΝΤΑΠΟΚΡΙΣΗΣ,/v1/unit?id=778764
...,...,...,...,...,...
63392,942218,21705,0,ΠΡΕΣΒΕΙΑ ΚΙΕΒΟΥ,/v1/unit?id=942218
63393,463876,29988,0,ΓΡΑΦΕΙΟ ΕΛΕΓΧΟΥ ΚΑΙ ΠΛΗΡΩΜΗΣ ΔΑΠΑΝΩΝ,/v1/unit?id=463876
63394,739483,99583,0,ΔΙΕΥΘΥΝΣΗ ΔΙΟΙΚΗΤΙΚΟΥ-ΟΙΚΟΝΟΜΙΚΟΥ,/v1/unit?id=739483
63395,524822,63432,0,ΥΠΗΡΕΣΙΕΣ ΥΠΑΓΟΜΕΝΕΣ ΑΠΕΥΘΕΙΑΣ ΣΤΟ ΔΗΜΑΡΧΟ,/v1/unit?id=524822


# Save Data

In [15]:
import pandas as pd
import csv
import numpy as np
import io
import os
import tempfile

def upload_csv_to_target_folder(name, dataframe):

  path = name+".csv"

  dataframe.to_csv(path, index=False)

  url=folder_analysis_url_shrpt+"json"
  target_folder = ctx.web.get_folder_by_server_relative_url(url)

  with open(path, "rb") as content_file:
      file_content = content_file.read()
      target_folder.upload_file(os.path.basename(path), file_content).execute_query()

In [16]:
upload_csv_to_target_folder("orgs", orgs)

In [21]:
upload_csv_to_target_folder("output_registries", output_registries)

In [31]:
upload_csv_to_target_folder("provision_org", provision_org)

In [37]:
upload_csv_to_target_folder("evidence_type", evidence_type)

In [58]:
upload_csv_to_target_folder("provision_org_owner_directory", provision_org_owner_directory)